# Notebook for text extraction on image
Inga Ulusoy, SSC, July 2022

In [ ]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    !pip install setuptools==61 -qqq
    # install misinformation
    !pip install git+https://github.com/ssciwr/misinformation.git -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

In [ ]:
import os
from IPython.display import Image, display
import misinformation

# download the models if they are not there yet
!python -m spacy download en_core_web_md
!python -m textblob.download_corpora

In [ ]:
images = misinformation.find_files(path="../data/images-little-text/", limit=1000)

In [ ]:
for i in images[0:3]:
    display(Image(filename=i))

In [ ]:
mydict = misinformation.utils.initialize_dict(images[0:10])

# google cloud vision API
First 1000 images per month are free.

In [ ]:
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "../data/misinformation-campaign-981aa55a3b13.json"

## Inspect the elements per image

In [ ]:
misinformation.explore_analysis(mydict, identify="text-on-image")

## Or directly analyze for further processing

In [ ]:
for key in mydict:
    print(key)
    mydict[key] = misinformation.text.TextDetector(
        mydict[key], analyse_text=True
    ).analyse_image()

## Convert to dataframe and write csv

In [ ]:
outdict = misinformation.utils.append_data_to_dict(mydict)
df = misinformation.utils.dump_df(outdict)

In [ ]:
# check the dataframe
df.head(10)

In [ ]:
# Write the csv
df.to_csv("drive/MyDrive/misinformation-data/data_out.csv")

## Topic analysis visual output

In [ ]:
import spacy
from bertopic import BERTopic

In [ ]:
# make a list of all the text_english entries per analysed image
list_keys = list(mydict.keys())
list_text_english = []
for key in mydict.keys():
    list_text_english.append(mydict[key]["text_english"])

In [ ]:
print(list_keys)

In [ ]:
all_texts = "".join(list_text_english)
print(list_text_english)

In [ ]:
nlp = spacy.load(
    "en_core_web_md",
    exclude=["tagger", "parser", "ner", "attribute_ruler", "lemmatizer"],
)

topic_model = BERTopic(embedding_model=nlp)
topics, probs = topic_model.fit_transform(list_text_english)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(0)